<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/HW3_%E5%AD%B8%E7%BF%92%E5%B9%B3%E5%8F%B0%E5%8A%A0%E7%95%AA%E8%8C%84%E9%90%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#試算表連結：https://docs.google.com/spreadsheets/d/1-B0zOFNoIppKXcHCIDQCWUK5Br3u8bRVnzvb-_Hsi_E/edit?gid=825292699#gid=825292699

!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 gradio pandas plotly

import os
import time
import uuid
import copy # 引入 copy 模組用於可靠的狀態管理
import pandas as pd
import gradio as gr
import plotly.express as px
from datetime import datetime
from dateutil.tz import gettz
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default

# Google Sheets 認證
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
except Exception as e:
    print(f"Google 授權失敗，請確保在 Colab 中運行授權單元格: {e}")
    class MockGSpreadClient:
        def open_by_url(self, url): return MockSheet()
    class MockSheet:
        def worksheet(self, title): return MockWorksheet()
        def add_worksheet(self, title, rows, cols): return MockWorksheet()
    gc = MockGSpreadClient()

# Google Sheet 設定
# 請使用您的實際 URL
SHEET_URL = "https://docs.google.com/spreadsheets/d/1-B0zOFNoIppKXcHCIDQCWUK5Br3u8bRVnzvb-_Hsi_E/edit?gid=429854282#gid=429854282"
TIMEZONE = "Asia/Taipei"

# 表頭定義
TASKS_HEADER = ["id", "task", "status", "start_time", "end_time", "actual_min", "pomodoros", "created_at", "completed_at"]
LOGS_HEADER = ["log_id", "task_id", "phase", "start_ts", "end_ts", "minutes", "cycles"]


class MockWorksheet:
    """用於在無法連接 Google Sheet 時的 Mock 物件"""
    def get_all_values(self): return []
    def update(self, data): pass
    def clear(self): pass
    def update_cells(self, data): pass
    def get_values(self): return []

class MockSheet:
    def worksheet(self, title): return MockWorksheet()
    def add_worksheet(self, title, rows, cols): return MockWorksheet()


# 確保工作表存在並有正確表頭
def ensure_worksheet(sh, title, header):
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=len(header)+5)

    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

try:
    gsheets = gc.open_by_url(SHEET_URL)
    # 使用固定的工作表名稱
    ws_tasks = ensure_worksheet(gsheets, "tasks", TASKS_HEADER)
    ws_logs = ensure_worksheet(gsheets, "pomodoro_logs", LOGS_HEADER)
except Exception as e:
    print(f"無法連接或初始化 Google Sheet：{e}")
    ws_tasks = MockWorksheet()
    ws_logs = MockWorksheet()


# 讀取 DataFrame
def read_df(ws, header):
    try:
        df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
        if df is None or df.empty:
            return pd.DataFrame(columns=header)
        df = df.fillna("")
        for c in header:
            if c not in df.columns:
                df[c] = ""

        # 確保數字欄位類型正確
        if "actual_min" in df.columns:
            df["actual_min"] = pd.to_numeric(df["actual_min"], errors="coerce").fillna(0).astype(float)
        if "pomodoros" in df.columns:
            df["pomodoros"] = pd.to_numeric(df["pomodoros"], errors="coerce").fillna(0).astype(int)

        # 確保 Dropdown 相關欄位是 str
        if "status" in df.columns: df["status"] = df["status"].astype(str)
        if "task" in df.columns: df["task"] = df["task"].astype(str)
        if "id" in df.columns: df["id"] = df["id"].astype(str)

        return df[header]
    except Exception as e:
        print(f"讀取工作表失敗：{e}")
        return pd.DataFrame(columns=header)

# 寫入 DataFrame
def write_df(ws, df, header):
    try:
        if df.empty:
            ws.clear()
            ws.update([header])
            return
        df_out = df.copy()
        for c in df_out.columns:
            df_out[c] = df_out[c].astype(str)

        # 使用 set_with_dataframe 更穩定
        set_with_dataframe(ws, df_out, include_index=False, row=1, col=1, resize=True)
    except Exception as e:
        print(f"寫入工作表失敗：{e}")

# 初始化全域 DataFrame
tasks_df = read_df(ws_tasks, TASKS_HEADER)
logs_df = read_df(ws_logs, LOGS_HEADER)

# 任務選擇清單
def list_task_choices():
    global tasks_df
    if tasks_df.empty:
        return []

    choices = []
    for _, r in tasks_df.iterrows():
        # 確保欄位訪問安全
        status = str(r.get('status', ''))
        task = str(r.get('task', ''))
        task_id = str(r.get('id', ''))

        display_name = f"[{status}] {task} — {task_id}"
        choices.append((display_name, task_id))

    return choices


# 刷新資料 (返回 4 個值，用於更新兩個 Dropdown)
def refresh_all():
    global tasks_df, logs_df
    tasks_df = read_df(ws_tasks, TASKS_HEADER)
    logs_df = read_df(ws_logs, LOGS_HEADER)
    choices = list_task_choices()
    return tasks_df, logs_df, choices, choices

# 時區當前時間
def tznow():
    return datetime.now(gettz(TIMEZONE))

# 新增任務 (返回 4 個值)
def add_task(task):
    global tasks_df
    if not task.strip():
        current_choices = list_task_choices()
        return "⚠️ 任務名稱不能為空", tasks_df, current_choices, current_choices

    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "created_at": _now,
        "completed_at": ""
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)

    new_choices = list_task_choices()
    return f"✅ 已新增任務：{task.strip()}", tasks_df, new_choices, new_choices


# 刪除任務 (返回 4 個值)
def delete_task(task_id):
    global tasks_df, logs_df
    current_choices = list_task_choices()
    if not task_id:
        return "⚠️ 請選擇一個任務", tasks_df, current_choices, current_choices

    task_id = str(task_id)
    if task_id not in tasks_df["id"].values:
        return f"⚠️ 找不到任務 ID：{task_id}", tasks_df, current_choices, current_choices

    task_name = tasks_df[tasks_df["id"] == task_id]["task"].iloc[0]
    tasks_df = tasks_df[tasks_df["id"] != task_id].reset_index(drop=True)
    logs_df = logs_df[logs_df["task_id"] != task_id].reset_index(drop=True)

    try:
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
        write_df(ws_logs, logs_df, LOGS_HEADER)
        new_choices = list_task_choices()
        return f"✅ 已刪除任務：{task_name} (ID: {task_id})", tasks_df, new_choices, new_choices
    except Exception as e:
        print(f"刪除任務失敗：{e}")
        return f"⚠️ 刪除任務失敗：{e}", tasks_df, current_choices, current_choices


# === 番茄鐘功能 (使用 gr.State) ===

# 開始階段 (修正: 新增 active_sessions 參數並使用 copy.deepcopy，返回 3 個值)
def start_phase(task_id, phase, cycles, active_sessions):
    if not task_id:
        return "⚠️ 請先選擇任務", list_task_choices(), active_sessions

    task_id = str(task_id)

    # 執行深層複製，確保 Gradio 偵測到狀態改變
    new_sessions = copy.deepcopy(active_sessions)

    # 在副本上操作
    new_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles)
    }

    task_name = tasks_df[tasks_df["id"] == task_id]["task"].iloc[0] if task_id in tasks_df["id"].values else "未知任務"

    new_choices = list_task_choices()
    # 返回三個值: 訊息, 任務選擇清單, 更新後的 active_sessions
    return f"▶️ 已開始：{phase}（任務：{task_name}，ID：{task_id}）", new_choices, new_sessions


# 結束階段 (修正: 新增 active_sessions 參數並使用 copy.deepcopy，返回 3 個值)
def end_phase(task_id, completed, active_sessions):
    global tasks_df, logs_df

    new_choices = list_task_choices()

    if not task_id:
        return "⚠️ 請選擇一個任務", new_choices, active_sessions

    task_id = str(task_id)

    # 執行深層複製
    new_sessions = copy.deepcopy(active_sessions)

    # 檢查 active_sessions 副本
    if task_id not in new_sessions:
        return "⚠️ 尚未開始任何階段", new_choices, active_sessions

    sess = new_sessions.pop(task_id) # 從副本中移除 session

    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)

    # 記錄 Log
    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": sess["cycles"]
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)

    # 更新任務實際時間和番茄數
    if sess["phase"] == "work":
        work_logs = logs_df[(logs_df["task_id"] == task_id) & (logs_df["phase"] == "work")]
        total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
        pomos = int(round(total_min / 25.0))

        idx = tasks_df.index[tasks_df["id"] == task_id]
        if len(idx) > 0:
            tasks_df.loc[idx[0], "actual_min"] = total_min
            tasks_df.loc[idx[0], "pomodoros"] = pomos

            if completed:
                tasks_df.loc[idx[0], "status"] = "done"
                tasks_df.loc[idx[0], "completed_at"] = tznow().isoformat()

            write_df(ws_tasks, tasks_df, TASKS_HEADER)

    task_name = tasks_df[tasks_df["id"] == task_id]["task"].iloc[0] if task_id in tasks_df["id"].values else "未知任務"

    new_choices = list_task_choices()

    # 返回三個值: 訊息, 任務選擇清單, 更新後的 new_sessions
    if sess["phase"] == "work" and sess["cycles"] % 4 == 0:
        return f"⏹️ 番茄鐘完成！請休息 15 分鐘，記錄 {minutes} 分鐘（任務：{task_name}）", new_choices, new_sessions
    elif sess["phase"] == "work":
        return f"⏹️ 番茄鐘完成！請休息 5 分鐘，記錄 {minutes} 分鐘（任務：{task_name}）", new_choices, new_sessions

    return f"⏹️ 已結束：{sess['phase']}，記錄 {minutes} 分鐘（任務：{task_name}）", new_choices, new_sessions


# 匯出記錄
def export_records(format_type):
    global tasks_df, logs_df
    timestamp = tznow().strftime("%Y%m%d_%H%M%S")
    try:
        # ... (匯出邏輯不變)
        if format_type == "CSV":
            filename_tasks = f"tasks_{timestamp}.csv"
            filename_logs = f"logs_{timestamp}.csv"
            tasks_df.to_csv(filename_tasks, index=False)
            logs_df.to_csv(filename_logs, index=False)
            return f"✅ 已匯出至 {filename_tasks} 和 {filename_logs}"
        elif format_type == "JSON":
            filename_tasks = f"tasks_{timestamp}.json"
            filename_logs = f"logs_{timestamp}.json"
            tasks_df.to_json(filename_tasks, orient="records")
            logs_df.to_json(filename_logs, orient="records")
            return f"✅ 已匯出至 {filename_tasks} 和 {filename_logs}"
        return "⚠️ 無效的格式"
    except Exception as e:
        return f"⚠️ 匯出失敗：{e}"

# 匯入記錄 (返回 3 個值)
def import_records(file, format_type):
    global tasks_df, logs_df
    current_choices = list_task_choices()
    try:
        if not file:
            return "⚠️ 請上傳檔案", current_choices, current_choices

        if format_type == "CSV":
            df = pd.read_csv(file.name)
        elif format_type == "JSON":
            df = pd.read_json(file.name)
        else:
            return "⚠️ 無效的格式", current_choices, current_choices

        if set(TASKS_HEADER).issubset(df.columns):
            tasks_df = pd.concat([tasks_df, df[TASKS_HEADER]], ignore_index=True).drop_duplicates(subset=['id'], keep='last')
            write_df(ws_tasks, tasks_df, TASKS_HEADER)
            new_choices = list_task_choices()
            return "✅ 任務記錄匯入成功", new_choices, new_choices
        elif set(LOGS_HEADER).issubset(df.columns):
            logs_df = pd.concat([logs_df, df[LOGS_HEADER]], ignore_index=True).drop_duplicates(subset=['log_id'], keep='last')
            write_df(ws_logs, logs_df, LOGS_HEADER)
            new_choices = list_task_choices()
            return "✅ 番茄鐘記錄匯入成功", new_choices, new_choices

        return "⚠️ 檔案格式不符合", current_choices, current_choices
    except Exception as e:
        return f"⚠️ 匯入失敗：{e}", current_choices, current_choices

# 查詢記錄 (返回 4 個值)
def query_records(completed_filter, start_date, end_date):
    df = tasks_df.copy()
    current_choices = list_task_choices()

    if df.empty:
        return "⚠️ 無記錄", pd.DataFrame(), current_choices, current_choices

    # 1. 過濾完成狀態
    if completed_filter != "All":
        df = df[df["status"] == ("done" if completed_filter == "Completed" else "todo")]

    # 2. 日期過濾修正 (增加更嚴格的錯誤檢查)
    if start_date or end_date:
        try:
            # 安全地將 created_at 轉換為 datetime，errors='coerce' 將無效值轉為 NaT
            df["created_dt"] = pd.to_datetime(df["created_at"], errors='coerce')

            # 關鍵修正：將所有時間戳都轉換為 TIMEZONE 時區，然後取得其日期部分 (00:00:00)
            # 必須使用 .dt.tz_convert 才能處理帶時區的字串，或先移除時區再localize

            # 處理可能帶有時區的 created_at 欄位:
            # 1. 將所有 NaT 或無效值清除，以便後續處理
            df = df.dropna(subset=['created_dt'])

            # 2. 將 created_dt 轉換到 'Asia/Taipei' 時區，然後正規化為日期
            # 如果 created_dt 已經有時區，tz_localize 會失敗，因此先嘗試 tz_convert
            def safe_tz_normalize(ts):
                if pd.isna(ts): return pd.NaT
                # 如果是時區感知 (tz-aware)，則轉換
                if ts.tz is not None:
                    return ts.tz_convert(TIMEZONE).normalize()
                # 如果是時區非感知 (tz-naive)，則先 localize 再 normalize
                return ts.tz_localize(TIMEZONE).normalize()

            df['created_date_only'] = df['created_dt'].apply(safe_tz_normalize)

            # 將輸入日期解析為帶有時區的日期，並正規化
            if start_date:
                start = pd.to_datetime(start_date).tz_localize(TIMEZONE).normalize()
                df = df[df['created_date_only'] >= start]

            if end_date:
                end = pd.to_datetime(end_date).tz_localize(TIMEZONE).normalize()
                # 為了包含當天，我們通常比較 < end + 1天，但在normalize的日期比較中， <= end 即可
                df = df[df['created_date_only'] <= end]

            # 清理暫時欄位
            df = df.drop(columns=['created_dt', 'created_date_only'], errors='ignore')

        except Exception as e:
            # 捕捉錯誤並印出，以便除錯
            # print(f"日期處理時發生例外: {e}")
            return "⚠️ 日期格式錯誤，請使用 YYYY-MM-DD", pd.DataFrame(), current_choices, current_choices

    return "✅ 查詢完成", df, current_choices, current_choices


# Gradio 介面
with gr.Blocks(title="學習平台") as demo:
    gr.Markdown("# 📚 學習平台（Google Sheet + 番茄鐘）")

    # === 狀態管理元件 (核心修正) ===
    active_sessions_state = gr.State({})

    with gr.Row():
        btn_refresh = gr.Button("🔄 重新整理")

    with gr.Tab("任務管理"):
        with gr.Row():
            task = gr.Textbox(label="任務名稱", placeholder="學習 Python / 完成報告...")
            btn_add = gr.Button("➕ 新增任務")
            msg_add = gr.Markdown()
        with gr.Row():
            task_choice = gr.Dropdown(choices=list_task_choices(), label="選擇任務", interactive=True)
            btn_delete = gr.Button("🗑️ 刪除任務")
            msg_delete = gr.Markdown()
        grid_tasks = gr.Dataframe(value=tasks_df, label="任務清單", interactive=False)

    with gr.Tab("番茄鐘"):
        with gr.Row():
            sel_task = gr.Dropdown(choices=list_task_choices(), label="選擇任務", interactive=True)
            cycles = gr.Number(value=1, precision=0, label="番茄數")
        with gr.Row():
            btn_start_work = gr.Button("▶️ 開始工作 (25 Min)")
            btn_end_work = gr.Button("⏹️ 結束工作 (中斷)")
            btn_complete = gr.Button("✅ 標記完成並結束")
        with gr.Row():
            btn_start_break = gr.Button("🍵 開始休息 (5/15 Min)")
            btn_end_break = gr.Button("⏹️ 結束休息 (中斷)")
        msg_pomo = gr.Markdown()
        grid_logs = gr.Dataframe(value=logs_df, label="番茄鐘記錄", interactive=False)

    with gr.Tab("查詢"):
        with gr.Row():
            completed_filter = gr.Dropdown(choices=["All", "Completed", "Not Completed"], value="All", label="完成狀態")
            start_date = gr.Textbox(label="開始日期 (YYYY-MM-DD)", placeholder="2025-10-01")
            end_date = gr.Textbox(label="結束日期 (YYYY-MM-DD)\n", placeholder="2025-10-31")
            btn_query = gr.Button("🔍 查詢")
        msg_query = gr.Markdown()
        grid_query = gr.Dataframe(label="查詢結果")

    with gr.Tab("匯入/匯出"):
        with gr.Row():
            export_format = gr.Dropdown(choices=["CSV", "JSON"], value="CSV", label="匯出格式")
            btn_export = gr.Button("📤 匯出記錄")
            msg_export = gr.Markdown()
        with gr.Row():
            import_file = gr.File(label="匯入檔案")
            import_format = gr.Dropdown(choices=["CSV", "JSON"], value="CSV", label="匯入格式")
            btn_import = gr.Button("📥 匯入記錄")
            msg_import = gr.Markdown()

    # 綁定動作
    btn_refresh.click(refresh_all, outputs=[grid_tasks, grid_logs, task_choice, sel_task])

    btn_add.click(add_task, inputs=[task], outputs=[msg_add, grid_tasks, task_choice, sel_task])

    btn_delete.click(delete_task, inputs=[task_choice], outputs=[msg_delete, grid_tasks, task_choice, sel_task])

    # === 番茄鐘功能綁定 (核心修正：傳入 active_sessions_state 作為輸入/輸出，並新增輸出) ===

    # 1. 開始工作
    btn_start_work.click(
        start_phase,
        inputs=[sel_task, gr.State("work"), cycles, active_sessions_state],
        outputs=[msg_pomo, sel_task, active_sessions_state] # 輸出 3 個值
    )

    # 2. 結束工作 (中斷)
    btn_end_work.click(
        lambda x, y: end_phase(x, completed=False, active_sessions=y),
        inputs=[sel_task, active_sessions_state],
        outputs=[msg_pomo, sel_task, active_sessions_state] # 輸出 3 個值
    )

    # 3. 標記完成並結束
    btn_complete.click(
        lambda x, y: end_phase(x, completed=True, active_sessions=y),
        inputs=[sel_task, active_sessions_state],
        outputs=[msg_pomo, sel_task, active_sessions_state] # 輸出 3 個值
    )

    # 4. 開始休息
    btn_start_break.click(
        start_phase,
        inputs=[sel_task, gr.State("break"), cycles, active_sessions_state],
        outputs=[msg_pomo, sel_task, active_sessions_state] # 輸出 3 個值
    )

    # 5. 結束休息 (中斷)
    btn_end_break.click(
        lambda x, y: end_phase(x, completed=False, active_sessions=y),
        inputs=[sel_task, active_sessions_state],
        outputs=[msg_pomo, sel_task, active_sessions_state] # 輸出 3 個值
    )

    # =================================

    btn_query.click(query_records, inputs=[completed_filter, start_date, end_date], outputs=[msg_query, grid_query, task_choice, sel_task])

    btn_export.click(export_records, inputs=[export_format], outputs=[msg_export])
    # 匯入 (返回 3 個值)
    btn_import.click(import_records, inputs=[import_file, import_format], outputs=[msg_import, task_choice, sel_task])

    btn_visualize.click(visualize_data, outputs=[plot_output])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d30ac10e10edf7d1f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
